In [1]:
import os
import fnmatch
import shutil
import xml.etree.ElementTree as ET
import glob
from xml.dom import minidom
import re
import subprocess
from PIL import Image, ImageChops

In [2]:
def replaceAll(search, replace, files=[], reg = True, flags = re.I | re.M, no_titel = False):
    
    if not files:
        files = glob.glob("new//OEBPS/*.xhtml")
    for file in files:
             
        if file == "new//OEBPS/toc.xhtml":
            continue
        if no_titel:
            if (file == "new//OEBPS/toc.xhtml") or (file == "new//OEBPS/cover.xhtml") or file == "new//OEBPS/titel_und_titelei.xhtml" or file == "new//OEBPS/impressum.xhtml":
                print(file)
                continue
            
        content_new = ""
        with open (file, 'r' ) as f:
            content = f.read()
            
            if reg:
                content_new = re.sub(search, replace, content, flags = flags)
            else:
                content_new = content.replace(search, replace)

        with open (file, 'w' ) as f:
            f.write(content_new)

In [3]:
# paste error log here
error_string = """
WARNING: The link is empty    [OEBPS/Kapitel-3.xhtml]
WARNING: The linked resource 'image/vitamin_c_cc0.png' does not exist    [OEBPS/Kapitel-6.xhtml]
WARNING: The file OEBPS/font/Roboto-Regular.otf has a MIME type that does not match its extension    [OEBPS/content.opf]
WARNING: The file OEBPS/font/QuayItcT-Book.otf has a MIME type that does not match its extension    [OEBPS/content.opf]
WARNING: The file OEBPS/font/Aleo-Italic.otf has a MIME type that does not match its extension    [OEBPS/content.opf]
WARNING: The file OEBPS/font/Roboto-Light.otf has a MIME type that does not match its extension    [OEBPS/content.opf]
WARNING: The file OEBPS/font/QuayItcT-BookItalic.otf has a MIME type that does not match its extension    [OEBPS/content.opf]
WARNING: The file OEBPS/font/Aleo-Regular.otf has a MIME type that does not match its extension    [OEBPS/content.opf]
WARNING: The file OEBPS/font/LiberationSerif-Italic.otf has a MIME type that does not match its extension    [OEBPS/content.opf]
WARNING: The file OEBPS/font/PermanentMarker-Regular.otf has a MIME type that does not match its extension    [OEBPS/content.opf]
WARNING: The file OEBPS/font/QuayItcT-Black.otf has a MIME type that does not match its extension    [OEBPS/content.opf]
WARNING: The file OEBPS/font/QuayItcT-BoldItalic.otf has a MIME type that does not match its extension    [OEBPS/content.opf]
WARNING: The file OEBPS/font/Aleo-Bold.otf has a MIME type that does not match its extension    [OEBPS/content.opf]
WARNING: The file OEBPS/font/QuayItcT-Bold.otf has a MIME type that does not match its extension    [OEBPS/content.opf]
"""

In [4]:
old_epub = "new.epub"
new_epub = "converted_new.epub"

# teufel hier werden mit +1 oder ohne 1 ersetzt
fix_teufel = ["k2_stern.png", "k1_engel.png", "k3_skater.png"]

In [5]:
old_zip = os.path.splitext(old_epub)[0] + ".zip"
new_zip = os.path.splitext(new_epub)[0] + ".zip"

shutil.copy(new_epub, new_zip)
shutil.copy(old_epub, old_zip)

# extract the epubs
os.system('unzip '+old_zip+' -d old')
os.system('unzip '+new_zip+' -d new')

os.remove(old_zip)
os.remove(new_zip)

In [6]:
# fix teufel

for teufel in fix_teufel:
    find=teufel
    replace = teufel
    
    t = teufel.split(".")
    
    # if teufel with 1 at end, remove the 1
    if t[0][-1] == "1":
        replace = t[0][:-1] + "."+t[1]
    else: # else add the 1
        replace = t[0] + "1" + "." +t[1]
    
    replaceAll(replace,find, reg=False)

# get all ressources that don't exist and copy to new epub

In [7]:
errors = error_string.splitlines()

In [8]:
opf = 'new/OEBPS/content.opf'

non_exist_files = []
with open (opf, 'r' ) as f:
    content = f.read()
    
    for error in errors:

        match = re.search(r"WARNING: The linked resource \'(.*?)\' does not exist",error)

        if match:
            file = match.group(1)
                           
            if file not in non_exist_files:

                non_exist_files.append(file)

                # copy the found recource from old to new

                # if png, convert to jpeg
                f = file.split('.')

                filetype = f[-1]
                if filetype.lower() == 'png':
                    
                    if file.split('/')[1] not in fix_teufel:
                        os.system('convert old/OEBPS/' + file + ' new/OEBPS/' + f[0] + '.jpg')
                        file_old = file
                        file = f[0] + '.jpg'
                        filetype = 'jpg'

                        replaceAll(file_old,file, reg=False)
                    else:
                        shutil.copy('old/OEBPS/'+file,'new/OEBPS/'+file)

                elif (filetype.lower() == 'jpg') or (filetype.lower() == 'jpeg'):
                    shutil.copy('old/'+file,'new/'+file)
                else:
                    print("WARNING: " + file + " is not an image")
                    continue


                # add the recources to content.opf
                if filetype.lower() == 'jpg':
                    filetype = 'jpeg'
                opf_image = '<item id="'+file.split('/')[1]+'" href="'+file+'" media-type="image/'+filetype+'"/>'
                content = re.sub('properties="cover-image"\s*/>', 'properties="cover-image"/>\n\t'+opf_image, content, flags = re.I | re.M)

        # remove all recources that are no longer referenced
        match = re.search(r"WARNING: The file (.*?) is not referenced",error)
        if match:
            file = match.group(1)
            os.remove('new/'+file)
            
            # remove the recources to content.opf
            f = file.split('.')
            filetype = f[-1]
            if filetype.lower() == 'jpg':
                    filetype = 'jpeg'
            
            file = file[6:]
            opf_image = '<item id="'+file.split('/')[-1]+'" href="'+file+'" media-type="image/'+filetype+'"/>'
            # content = re.sub(opf_image, '', content, flags = re.I | re.M)
            content = content.replace(opf_image, '')
            
            print(opf_image)

with open (opf, 'w' ) as f:
    f.write(content)

In [9]:
# crop all images so that no white border

for file in glob.glob('new/OEBPS/image/*.jpg'):
    
    im=Image.open(file)
    bg = Image.new(im.mode, im.size, (255,255,255))
    diff = ImageChops.difference(im, bg)
    #diff = ImageChops.add(diff, diff, 1.1, -5)
    imageBox = diff.getbbox()
    cropped = im.crop(imageBox)
    if cropped.size != im.size:
        cropped.save(file)

In [13]:
# create epub
p = subprocess.Popen(['zip','-rX', "../converted_"+new_epub,'mimetype','META-INF/','OEBPS/'], cwd='new/')
p.wait()

0

In [11]:
# cleanup
# shutil.rmtree('new/')
shutil.rmtree('old/')